<a href="https://colab.research.google.com/github/sowmyarshetty/NNClass/blob/main/LSTM_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m spacy download en_core_web_lg
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.0 MB/s eta 0:00:00


In [10]:
# Import spacy, import the large english model (this is 600 MB), and disable tagger, ner and lemmatizer.
# Download the large English model ~ 600 MB if you have not done so: `python -m spacy download en_core_web_lg`.
import spacy
nlp = spacy.load('en_core_web_lg',disable=["tagger", "ner", "lemmatizer"])

In [11]:
# Read in the text file with the read_file function.
def read_file(filepath):
    """
    Reads in a text file from the directory and saves the file contents to a variable.

    Args:
        text (str): The input text to read and save

    Returns:
        A string containing the file contents.
    """
    with open(filepath) as f:
        str_text = f.read()
    return str_text

## Tokenize and Clean Text

In [12]:
# Use the separate_punc function to remove the puncutation.
def separate_punc(md_text):
    """
    Retrieves all the words in the text without punctuation

    Args:
        text (str): The input text from which words are extracted witout punctuation.

    Returns:
        list: A list of words

    """
    # Create a list comprehension to get only the tokens, i.e., words in the text.
    return [token.text.lower() for token in nlp(md_text) \
            if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n']

In [13]:
# Pass in the first four chapters of Moby Dick to the read_file function.
md_text  = read_file('moby_dick_four_chapters.txt')


# Clean and tokenize the text using the separate_punc function.
tokens = separate_punc(md_text)

In [14]:
# Get the length of the tokens list.
len(tokens)

11338

In [15]:
# Look over the tokems to make sure all theo punctuation has been reomved.
# Some punctuation wasn't removed, i.e., "?--".
print(tokens[:300])

['call', 'me', 'ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long', 'precisely', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to', 'interest', 'me', 'on', 'shore', 'i', 'thought', 'i', 'would', 'sail', 'about', 'a', 'little', 'and', 'see', 'the', 'watery', 'part', 'of', 'the', 'world', 'it', 'is', 'a', 'way', 'i', 'have', 'of', 'driving', 'off', 'the', 'spleen', 'and', 'regulating', 'the', 'circulation', 'whenever', 'i', 'find', 'myself', 'growing', 'grim', 'about', 'the', 'mouth', 'whenever', 'it', 'is', 'a', 'damp', 'drizzly', 'november', 'in', 'my', 'soul', 'whenever', 'i', 'find', 'myself', 'involuntarily', 'pausing', 'before', 'coffin', 'warehouses', 'and', 'bringing', 'up', 'the', 'rear', 'of', 'every', 'funeral', 'i', 'meet', 'and', 'especially', 'whenever', 'my', 'hypos', 'get', 'such', 'an', 'upper', 'hand', 'of', 'me', 'that', 'it', 'requires', 'a', 'strong', 'moral', 'principle', 'to', 'prevent', 'me', 'from', '

In [16]:
# How many tokens contain "?--"?
count = 0
for token in tokens:
  if "?--" in token:
    count += 1

print(count)

6


The next step is to determine how long of token sequence we want to predict the next token.
This also affects our model's accuracy.
Hiakus are three token sequences and lyrics are seven token sequences.
We want to create 25 token sequences and then predict the next token, number 26.

## Create Sequences of Tokens

In [18]:
# Organize into sequences of tokens.
# Use 25 words to predict the 26th word using "+1".
train_len = 26 # = 25 training words plus one target word.

# Create an empty list of sequences
text_sequences = []

# Use a for loop to create lists of 26 tokens for the whole text.
for x in range(train_len,len(tokens)):

    # Range is 26 to 11,338
    # Get the train_len amount of characters
    sequence = tokens[x - train_len:x]

    # First iteration gets the words from 0:26
    # The second iteration getst the words from 1:27, etc.

    # Add to list of sequences
    text_sequences.append(sequence)




In [19]:
# The first 26 words [0:26]
print(text_sequences[0])
# The next 26 words [1:27]
print(text_sequences[1])
# The next 26 words [2:28].
print(text_sequences[2])

['call', 'me', 'ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long', 'precisely', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to', 'interest', 'me', 'on']
['me', 'ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long', 'precisely', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to', 'interest', 'me', 'on', 'shore']
['ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long', 'precisely', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to', 'interest', 'me', 'on', 'shore', 'i']


In [20]:
# Join the first 26 words.
print(' '.join(text_sequences[0]))
# Join the next 26 words.
print(' '.join(text_sequences[1]))
# And, join the next 26 words.
print(' '.join(text_sequences[2]))

call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on
me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore
ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i


In [21]:
# The list of text_sequences should be 26 less than the total tokens.
len(text_sequences)

11312

## Perform Tokenization with Keras

In [22]:
# Import the keras preprocessing text tokenizer from tensorflow to format the data from words into a numerical format.
from tensorflow.keras.preprocessing.text import Tokenizer

In [23]:
# Initialize the keras Tokenizer class
tokenizer = Tokenizer()
# Map each word with an index.
tokenizer.fit_on_texts(text_sequences)

In [24]:
# Get the dictionary mapping of words to their indices
print(tokenizer.word_index)

{'the': 1, 'a': 2, 'and': 3, 'of': 4, 'i': 5, 'to': 6, 'in': 7, 'it': 8, 'that': 9, 'he': 10, 'his': 11, 'was': 12, 'but': 13, 'me': 14, 'with': 15, 'as': 16, 'at': 17, 'this': 18, 'you': 19, 'is': 20, 'all': 21, 'for': 22, 'my': 23, 'on': 24, 'be': 25, "'s": 26, 'not': 27, 'from': 28, 'there': 29, 'one': 30, 'up': 31, 'what': 32, 'him': 33, 'so': 34, 'bed': 35, 'now': 36, 'about': 37, 'no': 38, 'into': 39, 'by': 40, 'were': 41, 'out': 42, 'or': 43, 'harpooneer': 44, 'had': 45, 'then': 46, 'have': 47, 'an': 48, 'upon': 49, 'little': 50, 'some': 51, 'old': 52, 'like': 53, 'if': 54, 'they': 55, 'would': 56, 'do': 57, 'over': 58, 'landlord': 59, 'thought': 60, 'room': 61, 'when': 62, 'could': 63, "n't": 64, 'night': 65, 'here': 66, 'head': 67, 'such': 68, 'which': 69, 'man': 70, 'did': 71, 'sea': 72, 'time': 73, 'other': 74, 'very': 75, 'go': 76, 'these': 77, 'more': 78, 'though': 79, 'first': 80, 'sort': 81, 'said': 82, 'last': 83, 'down': 84, 'most': 85, 'been': 86, 'never': 87, 'your':

In [26]:
# Get the dictionary of words and the number of times they appear in the text.
print(tokenizer.word_counts)

OrderedDict([('call', 27), ('me', 2471), ('ishmael', 133), ('some', 758), ('years', 135), ('ago', 84), ('never', 449), ('mind', 164), ('how', 321), ('long', 374), ('precisely', 37), ('having', 142), ('little', 767), ('or', 950), ('no', 1003), ('money', 120), ('in', 5647), ('my', 1786), ('purse', 71), ('and', 9646), ('nothing', 281), ('particular', 152), ('to', 6497), ('interest', 24), ('on', 1716), ('shore', 26), ('i', 7150), ('thought', 676), ('would', 702), ('sail', 104), ('about', 1014), ('a', 10377), ('see', 416), ('the', 15540), ('watery', 26), ('part', 234), ('of', 8287), ('world', 234), ('it', 4238), ('is', 1950), ('way', 390), ('have', 806), ('driving', 26), ('off', 416), ('spleen', 26), ('regulating', 26), ('circulation', 26), ('whenever', 130), ('find', 78), ('myself', 416), ('growing', 26), ('grim', 26), ('mouth', 130), ('damp', 78), ('drizzly', 26), ('november', 26), ('soul', 78), ('involuntarily', 52), ('pausing', 52), ('before', 364), ('coffin', 104), ('warehouses', 52), 

In [27]:
# What is the size of the vocabulary
vocabulary_size = len(tokenizer.word_counts)
print(vocabulary_size)


2718


In [28]:
# Encode each word in the text_sequences to the indices.
sequences = tokenizer.texts_to_sequences(text_sequences)

In [29]:
# Get the encoded indices for the the first 26 words
print(sequences[0])


[956, 14, 263, 51, 261, 408, 87, 219, 129, 111, 954, 260, 50, 43, 38, 314, 7, 23, 546, 3, 150, 259, 6, 2713, 14, 24]


In [30]:
# Get the word associated with the indices for the first sequence.
for i in sequences[0]:
  print(f"{i} - {tokenizer.index_word[i]}")

956 - call
14 - me
263 - ishmael
51 - some
261 - years
408 - ago
87 - never
219 - mind
129 - how
111 - long
954 - precisely
260 - having
50 - little
43 - or
38 - no
314 - money
7 - in
23 - my
546 - purse
3 - and
150 - nothing
259 - particular
6 - to
2713 - interest
14 - me
24 - on


In [34]:
# Get the word associated with a specific index.
tokenizer.index_word.get(956)

'call'

In [38]:
# Get the number of times the word "call" appears in the text.
count  = 0

index_to_word = {index : word for word, index in tokenizer.word_index.items()}
print(index_to_word)
for sequence in sequences:
  words = [index_to_word.get(index,"") for index in sequence]
  count += words.count(tokenizer.index_word.get(956,""))

print(count)

{1: 'the', 2: 'a', 3: 'and', 4: 'of', 5: 'i', 6: 'to', 7: 'in', 8: 'it', 9: 'that', 10: 'he', 11: 'his', 12: 'was', 13: 'but', 14: 'me', 15: 'with', 16: 'as', 17: 'at', 18: 'this', 19: 'you', 20: 'is', 21: 'all', 22: 'for', 23: 'my', 24: 'on', 25: 'be', 26: "'s", 27: 'not', 28: 'from', 29: 'there', 30: 'one', 31: 'up', 32: 'what', 33: 'him', 34: 'so', 35: 'bed', 36: 'now', 37: 'about', 38: 'no', 39: 'into', 40: 'by', 41: 'were', 42: 'out', 43: 'or', 44: 'harpooneer', 45: 'had', 46: 'then', 47: 'have', 48: 'an', 49: 'upon', 50: 'little', 51: 'some', 52: 'old', 53: 'like', 54: 'if', 55: 'they', 56: 'would', 57: 'do', 58: 'over', 59: 'landlord', 60: 'thought', 61: 'room', 62: 'when', 63: 'could', 64: "n't", 65: 'night', 66: 'here', 67: 'head', 68: 'such', 69: 'which', 70: 'man', 71: 'did', 72: 'sea', 73: 'time', 74: 'other', 75: 'very', 76: 'go', 77: 'these', 78: 'more', 79: 'though', 80: 'first', 81: 'sort', 82: 'said', 83: 'last', 84: 'down', 85: 'most', 86: 'been', 87: 'never', 88: 'yo

## Convert the List of Sequences to Arrays.

In [40]:
# Import numpy to convert the list of sequences to arrays.
import numpy as np

In [41]:
# Convert the all 26 word list of lists to arrays.
num_sequences = np.array(sequences)
print(num_sequences)

[[ 956   14  263 ... 2713   14   24]
 [  14  263   51 ...   14   24  957]
 [ 263   51  261 ...   24  957    5]
 ...
 [ 952   12  166 ...  262   53    2]
 [  12  166 2712 ...   53    2 2718]
 [ 166 2712    3 ...    2 2718   26]]


In [42]:
# Get the length of the array
len(num_sequences)

11312

In [43]:
# Get the first array.
print(num_sequences[0])


[ 956   14  263   51  261  408   87  219  129  111  954  260   50   43
   38  314    7   23  546    3  150  259    6 2713   14   24]


## Create input sequences and one-hot encode the target variable.

In [44]:
# Import the to_categorical function to convert the arrays to binary values.
import keras
from keras.utils import to_categorical

In [45]:
# Get the first 25 numbers from each array. These will be our "X".
# The -1 is to not include the 26th word which will be predicted i.e our y value
for sequence in num_sequences[:,:-1]:
  print(sequence)


Streaming output truncated to the last 5000 lines.
[ 170    1 2354    1  379 2355    3   21    1 2356  835   41   75 2357
   34    9    5   60   18  196  108  128    2   75 2358]
[   1 2354    1  379 2355    3   21    1 2356  835   41   75 2357   34
    9    5   60   18  196  108  128    2   75 2358   50]
[2354    1  379 2355    3   21    1 2356  835   41   75 2357   34    9
    5   60   18  196  108  128    2   75 2358   50 2359]
[   1  379 2355    3   21    1 2356  835   41   75 2357   34    9    5
   60   18  196  108  128    2   75 2358   50 2359   43]
[ 379 2355    3   21    1 2356  835   41   75 2357   34    9    5   60
   18  196  108  128    2   75 2358   50 2359   43 2360]
[2355    3   21    1 2356  835   41   75 2357   34    9    5   60   18
  196  108  128    2   75 2358   50 2359   43 2360   22]
[   3   21    1 2356  835   41   75 2357   34    9    5   60   18  196
  108  128    2   75 2358   50 2359   43 2360   22   11]
[  21    1 2356  835   41   75 2357   34    9    5   

In [46]:
# Get the last number (number 26) from each array. These will be our "y".

for sequence in num_sequences[:,-1]:
  print(sequence)

24
957
5
60
5
56
315
37
2
50
3
94
1
958
164
4
1
165
8
20
2
103
5
47
4
959
95
1
960
3
961
1
962
264
5
409
96
963
964
37
1
265
264
8
20
2
410
965
966
7
23
411
264
5
409
96
548
549
112
316
550
3
967
31
1
968
4
197
969
5
970
3
317
264
23
971
113
68
48
972
130
4
14
9
8
551
2
412
973
974
6
975
14
28
976
977
39
1
318
3
978
979
552
26
980
95
46
5
413
8
266
73
6
113
6
72
16
166
16
5
121
18
20
23
981
22
982
3
983
15
2
984
985
986
987
122
49
11
414
5
415
151
6
1
198
29
20
150
988
7
18
54
55
13
267
8
136
21
268
7
104
416
51
73
43
74
989
75
553
1
131
990
152
1
554
15
14
29
36
20
88
991
319
4
1
992
993
114
40
994
16
555
995
40
996
997
998
999
8
15
220
1000
221
3
417
1
178
151
19
556
222
1001
1002
20
1
557
115
9
558
1003
20
559
40
1004
3
1005
40
1006
69
2
320
269
321
41
42
4
322
4
179
223
17
1
560
4
167
1007
29
1008
1
319
4
2
1009
561
562
76
28
1010
1011
6
1012
1013
3
28
563
40
1014
1015
32
57
19
1016
53
564
1017
21
418
1
224
199
565
49
565
4
1018
268
419
7
554
566
51
567
168
1
1019
51
568
49
1
1020


In [47]:
# Set the input variable, `X`, to the first 25 numbers of each array.
X = num_sequences[:,:-1]
# Set target variable, `y`, to the last number of each array.
y = num_sequences[:,-1]

In [48]:
# Get the shape of X
print(X.shape)
# Get the length of each sequence.
print(X.shape[1])

(11312, 25)
25


In [49]:
# Get the shape of y
print(y.shape)

(11312,)


In [50]:
# Next,one-hot encode the target variable to get transform each index to a binary value.
# We increase the vocabulary by 1 so we can predict the next word.
y = to_categorical(y,num_classes=vocabulary_size+1)

In [51]:
# Get the shape of y again.
# we changed from a list (column) to horizontal
print(y.shape)


(11312, 2719)


In [52]:
# Print the first 24 binary values in the first array.
# The index in for the last word in before transforming was "24", this is converted to a "1".
print(y[0,:25])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1.]


## Creating a LSTM  Model

In [ ]:
# Import the dependencies for LSTM model.
import keras
from keras import layers
from keras import ops
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

In [ ]:
def create_model(vocabulary_size, seq_len):
    """
    Create and compile an LSTM-based sequential model for text generation.

    Parameters:
    - vocabulary_size (int): The size of the vocabulary, i.e., the number of unique words in the text.
    - seq_len (int): The length of the input sequences, indicating the number of words in each sequence.

    Returns:
    - model (Sequential): A compiled Keras sequential model for text generation.

    Model Architecture:
    - Embedding Layer: Maps word indices to dense vectors.
    - LSTM Layer 1: 150 units, returns full sequence.
    - LSTM Layer 2: 150 units.
    - Dense Layer: 150 units, ReLU activation.
    - Output Layer: Dense layer with softmax activation for multi-class classification.

    Compilation:
    - Loss: Categorical crossentropy.
    - Optimizer: Adam.
    - Metrics: Accuracy.

    Usage Example:
    model = create_model(vocabulary_size=2718, seq_len=25)
    """
    model = keras.Sequential()
    model.add(keras.layers.Embedding(vocabulary_size, seq_len))
    model.add(keras.layers.LSTM(150, return_sequences=True))
    model.add(keras.layers.LSTM(150))
    model.add(keras.layers.Dense(150, activation='relu'))
    model.add(keras.layers.Dense(vocabulary_size, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # Create a test input with the correct shape
    test_input = ops.ones((1, seq_len))
    print(f'The shape of the test input: {test_input}')
    # Call the model on the test input
    y = model(test_input)
    print(f'The shape of the test output: {y}')
    # Generate a summary of the model.
    model.summary()
    return model

## Training the Model

In [ ]:
# Define the model and pass in the vocabulary (+1) and the seq_len (25 words).
model = create_model(vocabulary_size+1, seq_len)

The shape of the test input: [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1.]]
The shape of the test output: [[0.00036833 0.00036808 0.00036676 ... 0.00036781 0.00036762 0.00036798]]


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (1, 25, 25)            │        67,975 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (1, 25, 150)           │       105,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (1, 150)               │       180,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (1, 150)               │        22,650 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (1, 2719)              │       410,569 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 787,394 (3.00 MB)

 Trainable params: 787,394 (3.00 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Fit model with 300 epochs. (This will take ~60 min to complete.)
model.fit(X, y, batch_size=128, epochs=300,verbose=1)

Epoch 1/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 17s 155ms/step - accuracy: 0.0445 - loss: 7.1666
Epoch 2/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 16s 176ms/step - accuracy: 0.0513 - loss: 6.3644
Epoch 3/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 14s 159ms/step - accuracy: 0.0531 - loss: 6.2633
Epoch 4/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 15s 169ms/step - accuracy: 0.0538 - loss: 6.1791
Epoch 5/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 15s 168ms/step - accuracy: 0.0575 - loss: 6.0032
Epoch 6/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 14s 156ms/step - accuracy: 0.0658 - loss: 5.8889
Epoch 7/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 15s 170ms/step - accuracy: 0.0666 - loss: 5.7632
Epoch 8/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 15s 166ms/step - accuracy: 0.0730 - loss: 5.7093
Epoch 9/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 14s 159ms/step - accuracy: 0.0737 - loss: 5.6617
Epoch 10/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 14s 157ms/step - accuracy: 0.0750 - loss: 5.5810
Epoch 11/300
89/89 ━━━━━━━━━━━━━━━━━━━━ 15s 164ms/step - accuracy: 0.0767 - loss: 5.4885
Epoch 12/300
89/89 ━━━━━━━━━━━

In [ ]:
from pickle import dump

In [ ]:
# Save the model to file
model.save('four_chapters_moby_dick_model_300.keras')
# Save the tokenizer
dump(tokenizer, open('four_chapters_moby_dick_tokenizer_300', 'wb'))

## Generating New Text

In [ ]:
# Import the dependencies needed for the LSTM.
from random import randint
from pickle import load
from keras.models import load_model
# You may need to pip install Keras-Preprocessing
from keras_preprocessing.sequence import pad_sequences

In [ ]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    '''
    Generate text using a trained language model.

    INPUTS:
     - model: Trained language model (e.g., LSTM) capable of text generation.
     - tokenizer: Tokenizer that was fit on text data.
     - seq_len: Length of the training sequences used to train the model.
     - seed_text: A raw string text serving as the seed for text generation.
     - num_gen_words: The number of words to be generated by model.
    '''

    # Final Output
    output_text = []

    # Intial Seed Sequence
    input_text = seed_text

    # Create num_gen_words
    for i in range(num_gen_words):

        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        # Pad sequences to our trained rate of 25 words.
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        # Predict Class Probabilities for each word
        pred_w = model.predict(pad_encoded, verbose=0)[0]

        pred_word_ind = np.argmax(pred_w, axis= -1)

        # Grab word
        pred_word = tokenizer.index_word[pred_word_ind]

        # Update the sequence of input text (shifting one over with the new word)
        input_text += ' ' + pred_word

        output_text.append(pred_word)

    # Make it look like a sentence.
    return ' '.join(output_text)

## Test: Grab a random seed sequence

In [ ]:
# Import the random module.
import random

In [ ]:
# Pick a random sequence of 26 words.

# Join the words


the mystery nay to this very hour i often puzzle myself with it now take away the awful fear and my sensations at feeling the supernatural


In [ ]:
# Import the load_model method.
from keras.models import load_model

In [ ]:
# You can set the model and tokenizer to the saved  model and tokenizer if you are satisified with the model's accuacy.
# Set the model to the saved trained 300 epoch model.
# model = load_model('four_chapters_moby_dick_model_300.keras')

# Set the tokenizer to the trained tokenizer from the model.
# tokenizer = load(open('four_chapters_moby_dick_tokenizer_300', 'rb'))

In [ ]:
# Call the generate_text function and pass in the required parameters. We set the num_gen_words = 25.
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=25)

"hand in mine were very similar in their strangeness to those which i experienced on waking up and seeing queequeg 's pagan arm thrown round"

- **The next 25 words aren't that accurate.**

## Explore Generating Text

In [ ]:
# Choose a 26 word text string from the first four chapters of Moby Dick.
# If less than 26 the accuracy is off.
text = """Seeing, now, that there were no curtains to the window, and that the street being very narrow,
the house opposite commanded a plain view into the room, and"""

In [ ]:
# Create tokens by using the separate_punc function.

# Join the tokens and set them to the "seed_text" variable.


['seeing', 'now', 'that', 'there', 'were', 'no', 'curtains', 'to', 'the', 'window', 'and', 'that', 'the', 'street', 'being', 'very', 'narrow', 'the', 'house', 'opposite', 'commanded', 'a', 'plain', 'view', 'into', 'the', 'room', 'and']
seeing now that there were no curtains to the window and that the street being very narrow the house opposite commanded a plain view into the room and


In [ ]:
# Call the generate_text function and pass in the required parameters. We set the num_gen_words =50.
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=25)

'observing more and more the indecorous figure that queequeg made staving about with little else but his hat and boots on i begged him as'

## Check the fist four chapters of Moby Dick to determine the accuracy of the text.
---
"Seeing, now, that there were no curtains to the window, and that the
street being very narrow, the house opposite commanded a plain view
into the room, and <font color='blue'>observing more and more the indecorous figure that
Queequeg made, staving about with little else but his hat and boots
on; I begged him as well as I could, to accelerate his toilet
somewhat, and particularly to get into his pantaloons as soon as
possible.</font>"

**Question: How would we gain better accuracy for the next 50 words?**

- Increase or decrease the length of the sequence?
- Decrease the batch size?